In [1]:
from astropy.table import Table
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [2]:
os.chdir('../../astro_codes/')
#import imp
#imp.reload(combining_catalogues)
import combining_catalogues
os.chdir('../Gas-SF/SF')

In [3]:
# create directory to save the made tables:
os.mkdir('matched_fits') if (os.path.isdir('matched_fits') == False) else None 

# Reference data:

In [4]:
gz_data = Table.read('../../fits/full_sample_debiased_w_low_z.fits')
print('Loaded GZ galaxies!')

Loaded GZ galaxies!


# NASA-Sloan atlas: FNugriz for low-z data

In [5]:
nsa_data = Table.read('../../fits/NSA/nsa_v0_1_2.fits')

nsa_matches = combining_catalogues.match_sky_restricted(gz_data,nsa_data,
                                                        reference_xyz=['RA', 'DEC', 'REDSHIFT_1'],
                                                        match_xyz=['RACAT', 'DECCAT', 'Z'])

nsa_matched = combining_catalogues.make_matched_catalogue(gz_data,nsa_data,nsa_matches)

248883 galaxies in the reference catalogue
145155 galaxies in the match catalogue
---> 145155 matches in total
---> 68629 unique matches of < 10 arcsec
---> 68482 unique matches of delta-z < 0.01


In [ ]:
def make_matched_catalogue(reference_data,match_data,ids):
    
    '''
    --- Create a catalogue of 'match' data that aligns perfectly with the reference
    catalogue---
    
    Inputs:
    -------
    reference_data: usually the catlogue we wish to match to (eg. galaxies in GZ).
    
    match_data: usually a subsidiary dataset, eg. detections in AFALFA, WISE, ...
    
    ids: an output from either match_sky(), restricted_match_sky() or match_ids().
    
    Outputs:
    --------
    match_table: table with the _columns_ of match data, matched to the reference 
    data catalogue. The 'mask' column provides simple access to whether the data 
    was matched or not.
    '''
    
    columns = match_data.colnames
    match_table = Table()
    
    mask = np.zeros(len(reference_data),dtype='bool')
    mask[ids['reference_index'].astype(int)] = True
    match_table['mask'] = mask
    
    for c in columns:
        if 'str' not in match_data[c].dtype.name: # only keep data which isn't a string!
            row1 = match_data[c][0].data
            # check if the item is a list:
            is_list = isinstance(row1,np.ndarray)
            if is_list:
                N_subarray = np.shape(row1)
                subarray_shape = (len(reference_data),) + N_subarray
                column_data = np.ones(subarray_shape)*(-999)
            else:
                column_data = np.ones(len(reference_data))*(-999)
            
            column_data[ids['reference_index'].astype(int)] = match_data[c][[ids['match_index'].astype(int)]]
            match_table[c] = column_data
            
    return match_table

In [ ]:
_ = make_matched_catalogue(gz_data,nsa_data,nsa_matches)

# MPA-JHU: SF from line emissions

In [6]:
mpa_data = Table.read('../../fits/MPA_JHU/classes.fits')
mpa_masses = Table.read('../../fits/MPA_JHU/masses.fits')
mpa_sfrs = Table.read('../../fits/MPA_JHU/sfrs.fits')

mpa_matches = combining_catalogues.match_ids(gz_data,mpa_data,'dr7objid_1','OBJID')

mpa_i_matched = combining_catalogues.make_matched_catalogue(gz_data,mpa_data,mpa_matches)
mpa_masses_matched = combining_catalogues.make_matched_catalogue(gz_data,mpa_masses,mpa_matches)
mpa_sfrs_matched = combining_catalogues.make_matched_catalogue(gz_data,mpa_sfrs,mpa_matches)

248883 galaxies in the reference catalogue
927552 galaxies in the match catalogue
---> 242480 matches in toal


# Chang et al. 2015: SDSS+WISE derived SFRs+masses

In [7]:
chang_input = Table.read('../../fits/Chang_15/sw_input.fits')
chang_output = Table.read('../../fits/Chang_15/sw_output.fits')

chang_matches = combining_catalogues.match_sky_restricted(gz_data,chang_input,
                                                          reference_xyz=['RA', 'DEC', 'REDSHIFT_1'],
                                                          match_xyz=['ra', 'dec', 'redshift'])

chang_input_matched = combining_catalogues.make_matched_catalogue(gz_data,chang_input,chang_matches)
chang_output_matched = combining_catalogues.make_matched_catalogue(gz_data,chang_output,chang_matches)

248883 galaxies in the reference catalogue
858365 galaxies in the match catalogue
---> 858365 matches in total
---> 245746 unique matches of < 10 arcsec
---> 245576 unique matches of delta-z < 0.01


# ALFALFA: gas fractions with SDSS survey overlap

In [8]:
a70 = Table.read('../../fits/ALFALFA/a70.csv')

# matches here are made in a separate location, in the 'Gas' folder.
a70_matches = Table.read('../sdss_alfa70_matches.fits')
a70_matches = a70_matches[a70_matches['i_alfa'] != -999]
a70_matches['reference_index'] = a70_matches['i_gz']
a70_matches['match_index'] = a70_matches['i_alfa']

a70_matched = combining_catalogues.make_matched_catalogue(gz_data,a70,a70_matches)

In [9]:
# Finally save all of the required files:

nsa_matched.write('matched_fits/nsa_matched.fits',overwrite=True)

mpa_i_matched.write('matched_fits/mpa_i_matched.fits',overwrite=True)
mpa_masses_matched.write('matched_fits/mpa_masses_matched.fits',overwrite=True)
mpa_sfrs_matched.write('matched_fits/mpa_sfrs_matched.fits',overwrite=True)

chang_input_matched.write('matched_fits/chang_input_matched.fits',overwrite=True)
chang_output_matched.write('matched_fits/chang_output_matched.fits',overwrite=True)

a70_matched.write('matched_fits/a70_matched.fits',overwrite=True)